In [1]:
__PRODUCTION__ = 1
__NAME__       = 'efficiency-ker'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 1.8

if __PRODUCTION__:
    from mplmagic import pgf
else:
    from mplmagic import svg

from mplmagic.util import SuperFigure, SuperAxes, size_hint
import matplotlib.pyplot as plt

print('This is how much space the figure will take up on letter paper')
size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import functools
import numpy as np
import colorsys
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import networkx as nx
import json
import pickle
import matplotlib

from symfac.experimental import RBFExpansionV2

In [3]:
d_tensor = torch.load('d1')
target = torch.tensor(d_tensor, dtype=torch.float)

<ipython-input-3-8c217515867a>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(d_tensor, dtype=torch.float)


In [5]:
gk_losses = pickle.loads(open('gk_losses_2.pickle', 'rb').read())
gkl_losses = pickle.loads(open('gk_long_losses.pickle', 'rb').read())


In [12]:
gkl_losses[0]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 40,
 60,
 80,
 100,
 120,
 140,
 160]

In [7]:
data = {
    'losses': {
        'svd': gk_losses[0],
        'rbf': gk_losses[1],
        'srbf': gk_losses[2][:10],
    },
    'times': {
        'rbf': [62/60, 98/60, 146/60, 169/60, 226/60, 250/60, 304/60, 310/60, 383/60, 413/60],
        'srbf': [63/60, 50/60, 52/60, 56/60, 50/60, 52/60, 58/60, 61/60, 64/60, 67/60],
    },
    'long_losses': {
        'svd': gkl_losses[1],
        'srbf': gkl_losses[0],
    },
}

open('data/efficiency.json', 'w').write(json.dumps(data))

994

In [30]:
[np.log(t) for t in gk_losses[0]]

[-5.101419232802069,
 -5.470150123627819,
 -5.879744297497574,
 -6.0196171385275745,
 -6.273456174231397,
 -6.64170550665747,
 -6.84750719863678,
 -7.02249295645256,
 -7.210618673429582,
 -7.291654609175741]

In [11]:
fig = SuperFigure(plt.figure(figsize=(__WIDTH__, __HEIGHT__), dpi=300))
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
x0 = 0.04
dx = 0.24
w = 0.22
axs = [
    fig.make_axes(
        left=x0 + dx * i,
        width=w,
        top=0.12,
        width_to_height=1.0,
        style='modern'
    ) for i in range(4)
]


def draw_bars(ax, h, offset, **kwargs):
    ax.bar(np.arange(len(h)) + offset, h, **kwargs)

bar_width = 0.4
bar_style = dict(
    width = 0.5*bar_width
)
svd_style = dict(
    color='r',
    label=r'\textbf{SVD}'
)
rbf_style = dict(
    color='k',
    label=r'\textbf{RBF}'
)
srbf_style = dict(
    color='g',
    label=r'\textbf{SRBF}'
)

axs[0].imshow(target)

axs[1].plot(gkl_losses[0], gkl_losses[1], 'g.-', gkl_losses[0], gkl_losses[2], 'r.-')    
axs[1].set_yscale('log')    

#draw_bars(axs[1], data['losses']['svd'], 0.5 * bar_width, **bar_style, **svd_style)
#draw_bars(axs[1], data['losses']['rbf'], 0.5 * bar_width + 0.6, **bar_style, **rbf_style)
#draw_bars(axs[1], data['losses']['srbf'], 0.5 * bar_width+0.3, **bar_style, **srbf_style)
axs[2].plot(range(1, 11), gk_losses[0], 'r.-', range(1, 11), gk_losses[1], 'k.-', range(1, 11), gk_losses[2][:10], 'g.-')
axs[2].set_yscale('log')  
axs[2].set_xbound(lower=0, upper=10)
axs[2].set_ybound(lower=0, upper=0.013)

draw_bars(axs[3], data['times']['srbf'], -0.5 * bar_width, **bar_style, **srbf_style)
draw_bars(axs[3], data['times']['rbf'], 0.5 * bar_width, **bar_style, **rbf_style)



title_style = dict(
    fontsize=8,
    y=0.975,
    va='bottom'
)

tick_style = dict(
    fontsize=7,
)
label_style = dict(
    fontsize=8,
    labelpad=0.75,
)

ticks = [0,250]
axs[0].set_xbound(lower=0, upper=250)
axs[0].set_ybound(lower=0, upper=250)
axs[0].set_xticks(ticks)
axs[0].set_yticks(ticks)
axs[0].set_xticklabels(['%d' % x for x in [0, 250]], **tick_style)
axs[0].set_yticklabels(['%d' % y for y in [0, 250]], **tick_style)


xticks = [1, 20, 40, 60, 80, 100, 120, 140, 160]
#yticks=[0,0.012]
axs[1].set_xbound(lower=-2, upper=160)
#axs[1].set_ybound(lower=0, upper=0.012)
axs[1].set_xticks(xticks)
#axs[1].set_yticks(yticks)
#axs[1].get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axs[1].set_xticklabels(['%d' % x for x in xticks], **tick_style)
axs[1].set_yticklabels([])

xticks = [1, 5, 10]
#yticks=[0,0.012]
axs[2].set_xbound(lower=0.7, upper=10.2)
#axs[2].set_ybound(upper=0.012)
axs[2].set_xticks(xticks)
#axs[2].set_yticks(yticks)
#axs[2].get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
axs[2].set_xticklabels(['%d' % x for x in xticks], **tick_style)
axs[2].set_yticklabels([])


xticks = [0,5,10]
yticks = [0,2,4,6]
axs[3].set_xbound(lower=0, upper=10)
axs[3].set_ybound(lower=0, upper=7)
axs[3].set_xticks(xticks)
axs[3].set_yticks(yticks)
axs[3].set_xticklabels(['%d' % x for x in [0,5,10]], **tick_style)
axs[3].set_yticklabels(['%d' % y for y in [0,2,4,6]], **tick_style)

axs[0].set_title(r'\textbf{Molecular kernel distance matrix}', **title_style)
axs[1].set_title(r'\textbf{MSE Losses}', **title_style)
axs[2].set_title(r'\textbf{MSE Losses}', **title_style)
axs[3].set_title(r'\textbf{Time comparison}', **title_style)

ax_canvas.bar([-1], [0], **bar_style, **svd_style)
ax_canvas.bar([-1], [0], **bar_style, **rbf_style)
ax_canvas.bar([-1], [0], **bar_style, **srbf_style)
ax_canvas.legend(
    loc='lower left',
    bbox_to_anchor=(0.5, 0.03),
    ncol=3,
    frameon=False,
    fontsize=6
)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

<ipython-input-11-a9d7e913f6d7>:55: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  axs[2].set_ybound(lower=0, upper=0.013)


In [12]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-efficiency-ker.pdf


In [80]:
gkl_losses[2] = [0.0115783,
 0.0070272,
 0.0042515,
 0.0036036,
 0.0030903,
 0.0027092,
 0.0024542,
 0.0022272,
 0.0020658,
 0.0019309,
 0.0018186,
 0.0017204,
 0.0016348,
 0.001561,
 0.0015032,
 0.0014486,
 0.0014016,
 0.0013568,
 0.0013175,
 0.0012819,
 0.0008705,
 0.0006629,
 0.0005288,
 0.0004283,
 0.0003465,
 0.0002765,
 0.0002148]

[0.0115783,
 0.0070272,
 0.0042515,
 0.0036036,
 0.0030903,
 0.0027092,
 0.0024542,
 0.0022272,
 0.0020658,
 0.0019309,
 0.0018186,
 0.0017204,
 0.0016348,
 0.001561,
 0.0015032,
 0.0014486,
 0.0014016,
 0.0013568,
 0.0013175,
 0.0012819,
 0.0008705,
 0.0006629,
 0.0005288,
 0.0004283,
 0.0003465,
 0.0002765,
 0.0002148]

In [82]:
gkl_losses[1]

[0.0061804,
 0.0046355,
 0.0038087,
 0.0030982,
 0.0021956,
 0.0021081,
 0.0014806,
 0.0015258,
 0.0010862,
 0.0010517,
 0.0009961,
 0.0009438,
 0.0008429,
 0.0007509,
 0.0006723,
 0.0006414,
 0.0007446,
 0.0006746,
 0.0006916,
 0.0006358,
 0.0002089,
 0.0001358,
 0.0001128,
 0.0001089,
 0.0001019,
 0.0001083,
 0.0001056]

END
---